In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
%%capture
!pip install netcdf4
!pip install geopandas

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.colors as colors
import glob as glb

In [ ]:
%cd /content/MyDrive/MyDrive/2023-1/3. Meteo fisica/Datos-rayos

/content/MyDrive/MyDrive/2023-1/3. Meteo fisica/Datos-rayos


In [ ]:
files = sorted(glb.glob("*dat"))
archivos=[]
for file in files:
   leer=pd.read_table(file, sep="\s+",header=None)
   leer.rename(columns={0:"year",1:"month",2:"day",3:"hour",4:"minute",
                        5:"second",7:"Y",8:"X",11:"Control",
                        12:"Polaridad"}, inplace=True )
   columnas=["year","month","day","hour","minute","second","Y","X",
             "Control","Polaridad"]
   df = leer[columnas]
   df = df[(df["Y"]>= -12.125)& (df["Y"]<= -11.625)]
   df = df[(df["X"]>= -76.125)& (df["X"]<= -75.625)]
   df = df[df["Control"]==0]
   df = df[(df["Polaridad"]== -3) | (df["Polaridad"]== 3)]
   archivos.append(df)
archivos

[      year  month  day  hour  minute  second         Y         X  Control  \
 89    2015      1    1    21      32      30 -11.73588 -76.01263        0   
 250   2015      1    2    19      52       1 -11.97078 -75.90048        0   
 378   2015      1    6    18      30      59 -11.81382 -75.99828        0   
 2855  2015      1   26    16      54      55 -11.91876 -75.70067        0   
 2871  2015      1   26    17      30      31 -11.83479 -76.01922        0   
 2879  2015      1   26    17      46       0 -11.78594 -75.79658        0   
 4153  2015      1   31    19      55      43 -11.65784 -76.03418        0   
 4163  2015      1   31    20       6      48 -11.65980 -76.05984        0   
 
       Polaridad  
 89           -3  
 250          -3  
 378          -3  
 2855          3  
 2871          3  
 2879          3  
 4153         -3  
 4163          3  ,
       year  month  day  hour  minute  second         Y         X  Control  \
 286   2015      2    4    20       5      29 

In [ ]:
combinaciones = []

# Realizar las combinaciones de los primeros 4 archivos
for i in range(0, 34, 2):
    comb = pd.merge(archivos[i], archivos[i+1], how="outer")
    combinaciones.append(comb)

# Realizar las combinaciones de las combinaciones anteriores
while len(combinaciones) > 1:
    new_combinaciones = []
    for i in range(0, len(combinaciones), 2):
        if i+1 < len(combinaciones):
            comb = pd.merge(combinaciones[i], combinaciones[i+1], how="outer")
            new_combinaciones.append(comb)
        else:
            # Agregar la combinación sin pareja (si hay un número impar de combinaciones)
            new_combinaciones.append(combinaciones[i])
    combinaciones = new_combinaciones

resultado_final = combinaciones[0]

In [ ]:
df_clim = pd.merge(resultado_final,archivos[34],how="outer")
df_clim.tail()

,year,month,day,hour,minute,second,Y,X,Control,Polaridad
2647,2017,11,29,19,57,45,-11.72079,-76.09520,0,-3
2648,2017,11,29,20,41,14,-11.81919,-76.07842,0,-3
2649,2017,11,29,21,23,24,-11.65403,-76.08765,0,-3
2650,2017,11,30,19,46,24,-11.69368,-75.79269,0,-3
2651,2017,11,30,19,56,38,-11.79359,-75.68331,0,3


In [ ]:
meses = ["enero", "febrero", "marzo","abril","mayo","junio","julio","agosto",
         "septiembre","octubre","noviembre","diciembre"]
df_por_mes = {}
for mes_numero, mes_nombre in enumerate(meses, start=1):
    df_por_mes[mes_nombre] = df_clim[df_clim["month"] == mes_numero]
for mes_nombre, df_por_mes in df_por_mes.items():
    globals()["df_" + mes_nombre] = df_por_mes

In [ ]:
lista_df = [df_enero,df_febrero,df_marzo,df_abril,df_mayo,df_junio,df_julio,
            df_agosto,df_septiembre,df_octubre,df_noviembre,df_diciembre]
lista_fechas = []
for i in lista_df:
  i["Fecha"]=pd.to_datetime(i[["year", "month", "day","hour","minute","second"]])
  i["light"]=1
  columnas=["Fecha","X","Y","light"]
  i=i[columnas]
  lista_fechas.append(i)

<ipython-input-9-f21c65e75798>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["Fecha"]=pd.to_datetime(i[["year", "month", "day","hour","minute","second"]])
<ipython-input-9-f21c65e75798>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["light"]=1
<ipython-input-9-f21c65e75798>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [ ]:
df_jan = lista_fechas[0]
df_feb = lista_fechas[1]
df_mar = lista_fechas[2]
df_apr = lista_fechas[3]
df_may = lista_fechas[4]
df_jun = lista_fechas[5]
df_jul = lista_fechas[6]
df_aug = lista_fechas[7]
df_sep = lista_fechas[8]
df_oct = lista_fechas[9]
df_nov = lista_fechas[10]
df_dic = lista_fechas[11]

## Por cuadriculas

In [ ]:
lista_aplicacion = [df_jan,df_feb,df_mar,df_apr,df_may,df_jun,df_jul,df_aug,
                    df_sep,df_oct,df_nov,df_dic]

clusters = []
for i in lista_aplicacion:
  #primer cuadrante: punto central (-76,-11.75)
  df_1 = i[(i["Y"]> -11.875)& (i["Y"]<= -11.625)]
  df_1 = df_1[(df_1["X"]> -76.125)& (df_1["X"]<= -75.875)]
  df_1 = df_1.set_index("Fecha")
  df1 = df_1.resample("60min").sum()
  df1["X"]= -76
  df1["Y"]= -11.75
  #segundo cuadrante: punto central (-75.75,-11.75)
  df_2 = i[(i["Y"]> -11.875)& (i["Y"]<= -11.625)]
  df_2 = df_2[(df_2["X"]> -75.875)& (df_2["X"]<= -75.625)]
  df_2 = df_2.set_index("Fecha")
  df2 = df_2.resample("60min").sum()
  df2["X"]= -75.75
  df2["Y"]= -11.75
  #tercer cuadrante: punto central (-76,-12)
  df_3 = i[(i["Y"]> -12.125)& (i["Y"]<= -11.875)]
  df_3 = df_3[(df_3["X"]> -76.125)& (df_3["X"]<= -75.875)]
  df_3 = df_3.set_index("Fecha")
  df3 = df_3.resample("60min").sum()
  df3["X"]= -76
  df3["Y"]= -12
  #cuarto cuadrante: punto central (-76,-11.75)
  df_4 = i[(i["Y"]> -12.125)& (i["Y"]<= -11.875)]
  df_4 = df_4[(df_4["X"]> -75.875)& (df_4["X"]<= -75.625)]
  df_4 = df_4.set_index("Fecha")
  df4 = df_4.resample("60min").sum()
  df4["X"]= -75.75
  df4["Y"]= -12

  #agrupando los cuadrantes
  df_total = pd.concat([df1,df2,df3,df4], axis=0).reset_index()
  df_total = df_total.sort_values(by = 'Fecha')
  df_total.rename(columns={"X":"longitude","Y":"latitude","Fecha":"time"},
                  inplace=True )
  #guardando
  clusters.append(df_total)

In [ ]:
enero_agrupado = clusters[0]
febrero_agrupado = clusters[1]
marzo_agrupado = clusters[2]
abril_agrupado = clusters[3]
mayo_agrupado = clusters[4]
junio_agrupado = clusters[5]
julio_agrupado = clusters[6]
agosto_agrupado = clusters[7]
septiembre_agrupado = clusters[8]
octubre_agrupado = clusters[9]
noviembre_agrupado = clusters[10]
diciembre_agrupado = clusters[11]

# Indice Cloud height

In [ ]:
cloud_base_Junin = xr.open_dataset("Datasets/Cloud_base_height_Junin.nc")
cloud_base_Junin

<xarray.Dataset>
Dimensions:    (longitude: 13, latitude: 8, time: 26304)
Coordinates:
  * longitude  (longitude) float32 -76.5 -76.25 -76.0 ... -74.0 -73.75 -73.5
  * latitude   (latitude) float32 -10.75 -11.0 -11.25 ... -12.0 -12.25 -12.5
  * time       (time) datetime64[ns] 2015-01-01 ... 2017-12-31T23:00:00
Data variables:
    cbh        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-04 23:53:47 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [ ]:
cloud_base_Junin = xr.open_dataset("Datasets/Cloud_base_height_Junin.nc")
cloud_base_prueba = cloud_base_Junin.sel(latitude=slice(-11.75,-12),
                             longitude=slice(-76,-75.75))
cloud_base_prueba

<xarray.Dataset>
Dimensions:    (longitude: 2, latitude: 2, time: 26304)
Coordinates:
  * longitude  (longitude) float32 -76.0 -75.75
  * latitude   (latitude) float32 -11.75 -12.0
  * time       (time) datetime64[ns] 2015-01-01 ... 2017-12-31T23:00:00
Data variables:
    cbh        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-04 23:53:47 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [ ]:
cbh_enero = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 1)
cbh_febrero = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 2)
cbh_marzo = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 3)
cbh_abril = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 4)
cbh_mayo = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 5)
cbh_junio = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 6)
cbh_julio = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 7)
cbh_agosto = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 8)
cbh_sep = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 9)
cbh_octubre = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 10)
cbh_nov = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 11)
cbh_dic = cloud_base_prueba.sel(time=cloud_base_prueba.time.dt.month == 12)

In [ ]:
#enero
cbh_enero = cbh_enero.to_dataframe()
df_cbh_enero= cbh_enero.reset_index()
#febrero
cbh_febrero = cbh_febrero.to_dataframe()
df_cbh_febrero= cbh_febrero.reset_index()
#marzo
cbh_marzo = cbh_marzo.to_dataframe()
df_cbh_marzo= cbh_marzo.reset_index()
#abril
cbh_abril = cbh_abril.to_dataframe()
df_cbh_abril = cbh_abril.reset_index()
#mayo
cbh_mayo = cbh_mayo.to_dataframe()
df_cbh_mayo = cbh_mayo.reset_index()
#junio
cbh_junio = cbh_junio.to_dataframe()
df_cbh_junio =cbh_junio.reset_index()
#julio
cbh_julio = cbh_julio.to_dataframe()
df_cbh_julio= cbh_julio.reset_index()
#agosto
cbh_agosto = cbh_agosto.to_dataframe()
df_cbh_agosto = cbh_agosto.reset_index()
#septiembre
cbh_sep =cbh_sep.to_dataframe()
df_cbh_sep=cbh_sep.reset_index()
#octubre
cbh_octubre = cbh_octubre.to_dataframe()
df_cbh_octubre =cbh_octubre.reset_index()
#noviembre
cbh_nov = cbh_nov.to_dataframe()
df_cbh_nov= cbh_nov.reset_index()
#diciembre
cbh_dic = cbh_dic.to_dataframe()
df_cbh_dic = cbh_dic.reset_index()

# Correlaciones considerando todos los casos

In [ ]:
#uniendo enero
df_corr_enero = pd.merge(enero_agrupado,df_cbh_enero,how="outer")
df_corr_enero=df_corr_enero.dropna()
#uniendo febrero
df_corr_febrero = pd.merge(febrero_agrupado,df_cbh_febrero,how="outer")
df_corr_febrero=df_corr_febrero.dropna()
#uniendo marzo
df_corr_marzo = pd.merge(marzo_agrupado,df_cbh_marzo,how="outer")
df_corr_marzo=df_corr_marzo.dropna()
#uniendo abril
df_corr_abril = pd.merge(abril_agrupado,df_cbh_abril,how="outer")
df_corr_abril= df_corr_abril.dropna()
#uniendo mayo
df_corr_mayo = pd.merge(mayo_agrupado,df_cbh_mayo,how="outer")
#uniendo junio
df_corr_junio = pd.merge(junio_agrupado,df_cbh_junio,how="outer")
df_corr_junio=df_corr_junio.dropna()
#uniendo julio
df_corr_julio = pd.merge(julio_agrupado,df_cbh_julio,how="outer")
df_corr_julio=df_corr_julio.dropna()
#uniendo agosto
df_corr_agosto = pd.merge(agosto_agrupado,df_cbh_agosto,how="outer")
df_corr_agosto=df_corr_agosto.dropna()
#uniendo septiembre
df_corr_septiembre = pd.merge(septiembre_agrupado,df_cbh_sep,how="outer")
df_corr_septiembre=df_corr_septiembre.dropna()
#uniendo octubre
df_corr_octubre = pd.merge(octubre_agrupado,df_cbh_octubre,how="outer")
#uniendo noviembre
df_corr_noviembre = pd.merge(noviembre_agrupado,df_cbh_nov,how="outer")
df_corr_noviembre=df_corr_noviembre.dropna()
#uniendo diciembre
df_corr_diciembre = pd.merge(diciembre_agrupado,df_cbh_dic,how="outer")
df_corr_diciembre=df_corr_diciembre.dropna()

## Correlaciones Spearman y Kendall: incluyendo dias sin rayos

In [ ]:
cbh_enero_spearman = df_corr_enero['cbh'].corr(df_corr_enero['light'], method='spearman')
cbh_febrero_spearman = df_corr_febrero['cbh'].corr(df_corr_febrero['light'], method='spearman')
cbh_marzo_spearman = df_corr_marzo['cbh'].corr(df_corr_marzo['light'], method='spearman')
cbh_abril_spearman = df_corr_abril['cbh'].corr(df_corr_abril['light'], method='spearman')
cbh_mayo_spearman = df_corr_mayo['cbh'].corr(df_corr_mayo['light'], method='spearman')
cbh_junio_spearman = df_corr_junio['cbh'].corr(df_corr_junio['light'], method='spearman')
cbh_julio_spearman = df_corr_julio['cbh'].corr(df_corr_julio['light'], method='spearman')
cbh_agosto_spearman = df_corr_agosto['cbh'].corr(df_corr_agosto['light'], method='spearman')
cbh_septiembre_spearman = df_corr_septiembre['cbh'].corr(df_corr_septiembre['light'], method='spearman')
cbh_octubre_spearman = df_corr_octubre['cbh'].corr(df_corr_octubre['light'], method='spearman')
cbh_noviembre_spearman = df_corr_noviembre['cbh'].corr(df_corr_noviembre['light'], method='spearman')
cbh_diciembre_spearman = df_corr_diciembre['cbh'].corr(df_corr_diciembre['light'], method='spearman')

In [ ]:
cbh_enero_kendall = df_corr_enero['cbh'].corr(df_corr_enero['light'], method='kendall')
cbh_febrero_kendall = df_corr_febrero['cbh'].corr(df_corr_febrero['light'], method='kendall')
cbh_marzo_kendall = df_corr_marzo['cbh'].corr(df_corr_marzo['light'], method='kendall')
cbh_abril_kendall = df_corr_abril['cbh'].corr(df_corr_abril['light'], method='kendall')
cbh_mayo_kendall = df_corr_mayo['cbh'].corr(df_corr_mayo['light'], method='kendall')
cbh_junio_kendall = df_corr_junio['cbh'].corr(df_corr_junio['light'], method='kendall')
cbh_julio_kendall = df_corr_julio['cbh'].corr(df_corr_julio['light'], method='kendall')
cbh_agosto_kendall = df_corr_agosto['cbh'].corr(df_corr_agosto['light'], method='kendall')
cbh_septiembre_kendall = df_corr_septiembre['cbh'].corr(df_corr_septiembre['light'], method='kendall')
cbh_octubre_kendall = df_corr_octubre['cbh'].corr(df_corr_octubre['light'], method='kendall')
cbh_noviembre_kendall = df_corr_noviembre['cbh'].corr(df_corr_noviembre['light'], method='kendall')
cbh_diciembre_kendall = df_corr_diciembre['cbh'].corr(df_corr_diciembre['light'], method='kendall')

In [ ]:
corr_spearman = [cbh_enero_spearman, cbh_febrero_spearman,cbh_marzo_spearman,
                 cbh_abril_spearman,cbh_mayo_spearman,cbh_junio_spearman,
                 cbh_julio_spearman,cbh_agosto_spearman,cbh_septiembre_spearman,
                 cbh_octubre_spearman,cbh_noviembre_spearman,cbh_diciembre_spearman]

In [ ]:
corr_kendall = [cbh_enero_kendall, cbh_febrero_kendall,cbh_marzo_kendall,
                cbh_abril_kendall,cbh_mayo_kendall,cbh_junio_kendall,
                cbh_julio_kendall,cbh_agosto_kendall,cbh_septiembre_kendall,
                cbh_octubre_kendall,cbh_noviembre_kendall,cbh_diciembre_kendall]

In [ ]:
data = {"Mes":[1,2,3,4,5,6,7,8,9,10,11,12],'Cbh-Spearman': corr_spearman,
        'Cbh-Kendall': corr_kendall}
df_correlaciones = pd.DataFrame(data)
df_correlaciones

,Mes,Cbh-Spearman,Cbh-Kendall
0,1,0.008632,0.007075
1,2,0.055791,0.045734
2,3,0.056659,0.046473
3,4,0.058198,0.047862
4,5,0.044748,0.036672
5,6,0.017765,0.014585
6,7,0.021694,0.017780
7,8,0.037320,0.030543
8,9,0.022529,0.018483
9,10,0.045783,0.037561


## Correlaciones Spearman y Kendall: no incluyendo dias sin rayos

In [ ]:
df_corr_enero1 = df_corr_enero[df_corr_enero["light"]>0]
df_corr_febrero1 = df_corr_febrero[df_corr_febrero["light"]>0]
df_corr_marzo1 = df_corr_marzo[df_corr_marzo["light"]>0]
df_corr_abril1 = df_corr_abril[df_corr_abril["light"]>0]
df_corr_mayo1 = df_corr_mayo[df_corr_mayo["light"]>0]
df_corr_junio1 = df_corr_junio[df_corr_junio["light"]>0]
df_corr_julio1 = df_corr_julio[df_corr_julio["light"]>0]
df_corr_agosto1 = df_corr_agosto[df_corr_agosto["light"]>0]
df_corr_septiembre1 = df_corr_septiembre[df_corr_septiembre["light"]>0]
df_corr_octubre1 = df_corr_octubre[df_corr_octubre["light"]>0]
df_corr_noviembre1 = df_corr_noviembre[df_corr_noviembre["light"]>0]
df_corr_diciembre1 = df_corr_diciembre[df_corr_diciembre["light"]>0]

In [ ]:
cbh_enero_spearman1 = df_corr_enero1['cbh'].corr(df_corr_enero1['light'], method='spearman')
cbh_febrero_spearman1 = df_corr_febrero1['cbh'].corr(df_corr_febrero1['light'], method='spearman')
cbh_marzo_spearman1 = df_corr_marzo1['cbh'].corr(df_corr_marzo1['light'], method='spearman')
cbh_abril_spearman1 = df_corr_abril1['cbh'].corr(df_corr_abril1['light'], method='spearman')
cbh_mayo_spearman1 = df_corr_mayo1['cbh'].corr(df_corr_mayo1['light'], method='spearman')
cbh_junio_spearman1 = df_corr_junio1['cbh'].corr(df_corr_junio1['light'], method='spearman')
cbh_julio_spearman1 = df_corr_julio1['cbh'].corr(df_corr_julio1['light'], method='spearman')
cbh_agosto_spearman1 = df_corr_agosto1['cbh'].corr(df_corr_agosto1['light'], method='spearman')
cbh_septiembre_spearman1 = df_corr_septiembre1['cbh'].corr(df_corr_septiembre1['light'], method='spearman')
cbh_octubre_spearman1 = df_corr_octubre1['cbh'].corr(df_corr_octubre1['light'], method='spearman')
cbh_noviembre_spearman1 = df_corr_noviembre1['cbh'].corr(df_corr_noviembre1['light'], method='spearman')
cbh_diciembre_spearman1 = df_corr_diciembre1['cbh'].corr(df_corr_diciembre1['light'], method='spearman')

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


In [ ]:
cbh_enero_kendall1 = df_corr_enero1['cbh'].corr(df_corr_enero1['light'], method='kendall')
cbh_febrero_kendall1 = df_corr_febrero1['cbh'].corr(df_corr_febrero1['light'], method='kendall')
cbh_marzo_kendall1 = df_corr_marzo1['cbh'].corr(df_corr_marzo1['light'], method='kendall')
cbh_abril_kendall1 = df_corr_abril1['cbh'].corr(df_corr_abril1['light'], method='kendall')
cbh_mayo_kendall1 = df_corr_mayo1['cbh'].corr(df_corr_mayo1['light'], method='kendall')
cbh_junio_kendall1 = df_corr_junio1['cbh'].corr(df_corr_junio1['light'], method='kendall')
cbh_julio_kendall1 = df_corr_julio1['cbh'].corr(df_corr_julio1['light'], method='kendall')
cbh_agosto_kendall1 = df_corr_agosto1['cbh'].corr(df_corr_agosto1['light'], method='kendall')
cbh_septiembre_kendall1 = df_corr_septiembre1['cbh'].corr(df_corr_septiembre1['light'], method='kendall')
cbh_octubre_kendall1 = df_corr_octubre1['cbh'].corr(df_corr_octubre1['light'], method='kendall')
cbh_noviembre_kendall1 = df_corr_noviembre1['cbh'].corr(df_corr_noviembre1['light'], method='kendall')
cbh_diciembre_kendall1 = df_corr_diciembre1['cbh'].corr(df_corr_diciembre1['light'], method='kendall')

In [ ]:
corr_spearman1 = [cbh_enero_spearman1, cbh_febrero_spearman1,cbh_marzo_spearman1,
                 cbh_abril_spearman1,cbh_mayo_spearman1,cbh_junio_spearman1,
                 cbh_julio_spearman1,cbh_agosto_spearman1,cbh_septiembre_spearman1,
                 cbh_octubre_spearman1,cbh_noviembre_spearman1,cbh_diciembre_spearman1]

corr_kendall1 = [cbh_enero_kendall1, cbh_febrero_kendall1,cbh_marzo_kendall1,
                cbh_abril_kendall1,cbh_mayo_kendall1,cbh_junio_kendall1,
                cbh_julio_kendall1,cbh_agosto_kendall1,cbh_septiembre_kendall1,
                cbh_octubre_kendall1,cbh_noviembre_kendall1,cbh_diciembre_kendall1]

In [ ]:
data1 = {"Mes":[1,2,3,4,5,6,7,8,9,10,11,12],'Cbh-Spearman_1': corr_spearman1,
        'Cbh-Kendall_1': corr_kendall1}
df_correlaciones1 = pd.DataFrame(data1)
df_correlaciones1

,Mes,Cbh-Spearman_1,Cbh-Kendall_1
0,1,-0.103944,-0.080009
1,2,0.089558,0.067757
2,3,0.084750,0.065535
3,4,0.080604,0.061585
4,5,-0.105299,-0.079528
5,6,0.394604,0.308697
6,7,NaN,NaN
7,8,-0.156853,-0.134236
8,9,0.032536,0.028600
9,10,0.119744,0.091011


In [ ]:
cbh_dataframe = pd.merge(df_correlaciones,df_correlaciones1,how="outer")
cbh_dataframe

,Mes,Cbh-Spearman,Cbh-Kendall,Cbh-Spearman_1,Cbh-Kendall_1
0,1,0.008632,0.007075,-0.103944,-0.080009
1,2,0.055791,0.045734,0.089558,0.067757
2,3,0.056659,0.046473,0.084750,0.065535
3,4,0.058198,0.047862,0.080604,0.061585
4,5,0.044748,0.036672,-0.105299,-0.079528
5,6,0.017765,0.014585,0.394604,0.308697
6,7,0.021694,0.017780,NaN,NaN
7,8,0.037320,0.030543,-0.156853,-0.134236
8,9,0.022529,0.018483,0.032536,0.028600
9,10,0.045783,0.037561,0.119744,0.091011


# Guardando

In [ ]:
df_cbh_correlaciones = pd.concat([df_corr_enero,df_corr_febrero,df_corr_marzo,
                                   df_corr_abril,df_corr_mayo,df_corr_junio,
                                   df_corr_julio, df_corr_agosto,df_corr_septiembre,
                                   df_corr_octubre, df_corr_noviembre, df_corr_diciembre])
df_cbh_correlaciones

,time,longitude,latitude,light,cbh
0,2015-01-01 21:00:00,-76.00,-11.75,1.0,68.567871
1,2015-01-01 22:00:00,-76.00,-11.75,0.0,29.947266
2,2015-01-01 23:00:00,-76.00,-11.75,0.0,29.947266
3,2015-01-02 00:00:00,-76.00,-11.75,0.0,44.671387
4,2015-01-02 01:00:00,-76.00,-11.75,0.0,56.257812
...,...,...,...,...,...
37380,2016-12-30 20:00:00,-75.75,-11.75,0.0,59.154297
37381,2016-12-30 20:00:00,-76.00,-11.75,1.0,70.740234
37382,2016-12-30 21:00:00,-76.00,-11.75,1.0,367.152832
37383,2016-12-30 21:00:00,-75.75,-11.75,0.0,401.669922


In [ ]:
df_cbh_correlaciones.to_csv("Scikit_regresion_multivariada/Cloud_base_height.csv")

In [ ]:
cbh_dataframe.to_csv("Correlaciones_rayos/Correlacion_Cbh.csv")